In [ ]:
library(dplyr)
library(class)
library(tidyverse)
library(pROC)
library(glmnet)
library(glmnetUtils)

install.packages("splitTools")
library(splitTools)

install.packages('multiROC')
library(multiROC)

install.packages('dummies')
library(dummies)

install.packages("doParallel")
library(doParallel)

install.packages("foreach")
library(foreach)

install.packages("doRNG")
library(doRNG)

In [ ]:
#ATAC_pred <- readRDS(snakemake@input[["input_predictions"]])
ATAC_pred <- readRDS("../data/ATAC_predictions_train_20_predict_80/ATAC_pred_lasso_normalized_trimmed_formatted_standardized.rds")

head(ATAC_pred)

In [ ]:
#sample_types <- read.table(snakemake@input[["input_sample_types"]], header = F, sep = " ")
sample_types <- read.table("../data/sample_types.txt", header = F, sep = " ")

colnames(sample_types) <- c("sample", "sample_type")
head(sample_types)

In [ ]:
data <- merge(ATAC_pred, sample_types, by="sample")
#data <- data %>% mutate(sample_type01 = ifelse(sample_type == "Healthy", 0, 1))
head(data)

In [ ]:
data %>% group_by(sample_type) %>% summarize(n = n())
data <-subset(data, sample_type != "Duodenal_Cancer")
data$sample_type <- as.factor(data$sample_type)
data <- data %>% droplevels("Duodenal_Cancer")
data %>% group_by(sample_type) %>% summarize(n = n())

In [ ]:
data <- data %>% select(-sample)
head(data)

In [ ]:


nested_CV_lasso <- function(data, k_inner_cv, k_outer_cv){
    
    y_all <- data %>% dplyr::select(sample_type) %>% as.matrix()
    classes <- unique(y_all)
    
    X <- data %>% dplyr::select(-sample_type) %>% as.matrix() 
    observed <- y_all
    return_tibble <- tibble(observed = rep(data$sample_type, k_outer_cv), 
                            CV_rep = rep(1:k_outer_cv, each=nrow(data)))
    
    cl <- makePSOCKcluster(5, outfile="")
    registerDoParallel(cl)
    return_tibble <- foreach(class = 1:length(unique(y_all)), 
                            .inorder = TRUE,
                            .options.RNG = 1985,
                            .combine = "cbind",
                            .packages = c("splitTools", "glmnetUtils")) %dorng% {
        
        get_cvm <- function(model) {
            index <- match(model$lambda.min, model$lambda)
            model$cvm[index]
        }
        
        y <- ifelse(y_all==classes[class], 1, 0)
        return_vector_for_class <- c()
        
        for (i in 1:k_outer_cv){ # repeated Cross-validation loop
            
            set.seed(i)
            #cvfolds <- cut(1:nrow(data), breaks = k_inner_cv, labels = F)
            folds <- create_folds(y, k = k_inner_cv)
            #cvfolds <- sample(cvfolds)

            predicted <- rep(NA, nrow(data))
                                
            for (fold in folds){
                testdata  <- X[-fold,]
                traindata <- X[fold,]
                train_y <- y[fold,]

                ########### nested CV to find best alpha and lambda on train folds ###########
                set.seed(0) # alpha
                lasso_cva <- cva.glmnet(traindata, train_y, nfolds = 3, family = "binomial")
                enet_performance <- data.frame(alpha = lasso_cva$alpha)
                models <- lasso_cva$modlist
                enet_performance$cvm <- vapply(models, get_cvm, numeric(1))
                minix <- which.min(enet_performance$cvm)
                best_alpha <- lasso_cva$alpha[minix]

                set.seed(0) # lambda
                lasso_cv <- cv.glmnet(traindata, 
                                      train_y, 
                                      alpha = best_alpha, 
                                      standardize = TRUE, 
                                      nfolds = 3, 
                                      family = "binomial")
                #message(plot(lasso_cv))
                lambda_cv <- lasso_cv$lambda.min
                
                cat("best_alpha\n", file=stdout())
                cat(best_alpha, file=stdout())
                message("best_alpha")
                message(best_alpha)
                message("best_lambda")
                message(lambda_cv)
                ####################################################################

                fit       <- glmnet(traindata, train_y, family = "binomial", alpha = best_alpha, lambda = lambda_cv)
                tmp       <- predict(fit, s=lambda_cv, testdata, type = "response")
                predicted[-fold] <- tmp
                }

            return_vector_for_class <- c(return_vector_for_class, predicted)
        }  # end of outer cv loop  
        tibble("{classes[class]}_pred" := return_vector_for_class)
    }
    stopCluster(cl)
    registerDoSEQ()
    
    return(return_tibble)
}

In [ ]:
k_outer_cv = 2
results <- nested_CV_lasso(data, k_inner_cv = 2, k_outer_cv = k_outer_cv)

In [ ]:
#results <- tibble(results, .name_repair = "unique")
head(results)

In [ ]:
saveRDS(results, file = snakemake@output[["lasso_output"]])

In [ ]:
for ( col in 1:ncol(results)){
    colnames(results)[col] <-  sub("_pred", "", colnames(results)[col])
}
head(results)

## AUC calculation with pROC package == One vs. one

In [ ]:
rocs <- list()
list_names <- paste0("CV_rep_", 1:k_outer_cv)
for (i in 1:k_outer_cv){
    res_CV <- results %>% filter(CV_rep == i) %>% select(-c(CV_rep, observed))
    res_CV <- 
    roc <- multiclass.roc(response = data$sample_type, predictor = res_CV)
    rocs[[i]] <- roc
}
names(rocs) <- c(list_names)

paste0("The AUC of the first CV repetition: ", rocs[[1]]$auc)

aucs <- c()
for (i in 1:length(rocs)){
    aucs <- c(aucs, rocs[[i]]$auc) 
}

paste0("The mean AUC of all the CV repetitions: ", mean(aucs))

# AUC calculation with multiROC package == One vs. rest, and plotting

#### Run the multiROC and plotting for all the CV repetitions

In [ ]:
Specificities <- NULL
Sensitivities <- NULL

for (i in 1:k_outer_cv){
    res_CV <- data.frame(results %>% filter(CV_rep == i) %>% select(-c(CV_rep, observed)))
    
    colnames(res_CV) <- paste(colnames(res_CV), "_pred_lasso", sep = "")

    true_label <- dummies::dummy(data$sample_type, sep = ".")
    true_label <- data.frame(true_label)
    colnames(true_label) <- gsub(".*?\\.", "", colnames(true_label))
    colnames(true_label) <- paste(colnames(true_label), "_true", sep = "")
    final_df <- cbind(true_label, res_CV)

    roc_res <- multi_roc(final_df, force_diag=T)
    
    if (i == 1){
        AUCs <- as.data.frame(t(unlist(roc_res$AUC)))
    }
    else {
        AUCs <- rbind(AUCs, as.data.frame(t(unlist(roc_res$AUC))))
    }
    
    plot_roc_df <- plot_roc_data(roc_res)
    plot_roc_df <- plot_roc_df %>% select(-Method)
    
    if (i == 1){
        Specificities <- plot_roc_df %>% select(Specificity)
        Sensitivities <- plot_roc_df %>% select(Sensitivity)
        Groups  <- plot_roc_df %>% select(Group)
    }
    else {
        Specificities <- cbind(Specificities, plot_roc_df %>% select(Specificity))
        Sensitivities <- cbind(Sensitivities, plot_roc_df %>% select(Sensitivity))
    }
    
    #IRdisplay::display(all_plot_roc_df)

}
Specificities <- tibble(Specificities, .name_repair = "unique")
Specificities <- Specificities %>% mutate(mean = rowMeans(across()))
Sensitivities <- tibble(Sensitivities, .name_repair = "unique")
Sensitivities <- Sensitivities %>% mutate(mean = rowMeans(across()))

(AUCs <- tibble(AUCs))

all_sensitivity_specificity <- tibble(Specificity = Specificities$mean, Sensitivity = Sensitivities$mean, Group = Groups$Group)
head(all_sensitivity_specificity)

In [ ]:
ggplot(all_sensitivity_specificity, aes(x = 1-Specificity, y=Sensitivity)) +
  geom_path(aes(color = Group), size=1) +
  geom_segment(aes(x = 0, y = 0, xend = 1, yend = 1), 
                        colour='grey', linetype = 'dotdash') +
  theme_bw() + 
  theme(plot.title = element_text(hjust = 0.5), 
                 legend.justification=c(1, 0), legend.position=c(.95, .05),
                 legend.title=element_blank(), 
                 legend.background = element_rect(fill=NULL, size=0.5, 
                                                           linetype="solid", colour ="black"))

In [ ]:
print("Mean AUC over 10 repetitions of 10-fold CV: ")
(mean_AUCs <- colMeans(AUCs[sapply(AUCs, is.numeric)]))

In [ ]:
results01 <- tibble()

for (i in 1:k_outer_cv){
    res_CV <- results %>% filter(CV_rep == i)
    obs_rep <- res_CV %>% select(c(CV_rep, observed))
    res_CV <- res_CV %>% select(-c(CV_rep, observed))
    res_CV <- res_CV %>% mutate(pred01 = factor(colnames(res_CV)[apply(res_CV,1,which.max)], ordered = TRUE))
    res_CV <- cbind(res_CV, obs_rep)
    results01 <- rbind(results01, res_CV)
}

head(results01)

In [ ]:
error_rates <- c()
accuracies <- c()
for (i in 1:k_outer_cv){
    res_CV <- results01 %>% filter(CV_rep == i)
    error_rates <- c(error_rates, mean(as.character(res_CV$observed) != as.character(res_CV$pred01)))
    accuracies <- c(accuracies, mean(as.character(res_CV$observed) == as.character(res_CV$pred01)))
}

error_rates
accuracies

paste("Mean error rate over 10 repetitions of 10-fold CV: ", mean(error_rates), sep = "")
paste("Mean accuracy over 10 repetitions of 10-fold CV: ", mean(accuracies), sep = "")

### for one vs rest classifier: https://stats.stackexchange.com/questions/71700/how-to-draw-roc-curve-with-three-response-variable/110550#110550

In [ ]:
install.packages("ROCR")
library(ROCR)
install.packages("rlist")
library(rlist)

In [ ]:
data(ROCR.xval)
str(ROCR.xval)
predictions <- ROCR.xval$predictions

In [ ]:
list_res <- list(predictions = list(), labels = list())
str(list_res)

In [ ]:
classes <- unique(data$sample_type)

aucs = c()


plot(x=NA, y=NA, xlim=c(0,1), ylim=c(0,1),
     bty='n')
legend_text <- c()

for (i in 1:length(classes)) {
    for (rep in 1:k_outer_cv){
        res_CV <- results %>% filter(CV_rep == rep) %>% select(-c(observed, CV_rep)) 
        class <- paste0(classes[i], "")
        pred = pull(res_CV, class)
        list.append(list_res$predictions, rep = pred)
        obs <- ifelse(data$sample_type == class, 1, 0)
        list.append(list_res$labels, rep = obs)
        }
    pred = prediction(pred, obs)
    nbperf = performance(pred, "tpr", "fpr")
    
    roc.x = unlist(nbperf@x.values)
    roc.y = unlist(nbperf@y.values)
    par(new=TRUE)
    #lines(roc.y ~ roc.x, col=i+1, lwd=2)
    plot(nbperf,
     avg='threshold',
     lwd=3,
     col=i + 1)
    legend_text <- c(legend_text, class)


    nbauc = performance(pred, "auc")
    nbauc = unlist(slot(nbauc, "y.values"))
    aucs <- c(aucs, nbauc)
    }

lines(x=c(0,1), c(0,1))
mean(aucs)
aucs
legend('bottomright', legend = legend_text, lty = 1, col=2:(length(classes)+1))